In [163]:
import os
import sys
import pandas as pd # data frame kullanmak icin
import numpy as np  #sayisal verilere donusturmek icin
import seaborn as sns 
import matplotlib.pyplot as plt
import re # ingilizce kelimeleri kontrol etmek icin kullanidigim kutuphanedir
# nltk ingilizce kelimeleri split, uppercase, lowercase, stem, join etme icin kullanidigim kutuphanedir
from nltk.corpus import stopwords 
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score 
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error

In [164]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [165]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
print(stopwords.words('english')) #English Stopwords

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/erenbekman/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [166]:
news_data = pd.read_csv('train.csv')
news_data.head(15)
# ilk 15 degeri gösterdim
# 1 : Fake News, 0 : Real News

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi...",0
6,6,Life: Life Of Luxury: Elton John’s 6 Favorite ...,NaN,Ever wonder how Britain’s most iconic pop pian...,1
7,7,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi...",0
8,8,Excerpts From a Draft Script for Donald Trump’...,NaN,Donald J. Trump is scheduled to make a highly ...,0
9,9,"A Back-Channel Plan for Ukraine and Russia, Co...",Megan Twohey and Scott Shane,A week before Michael T. Flynn resigned as nat...,0


Basic Data Analysis

In [167]:
news_data.shape # Veri setimiz 20800 satır , 5 sütundan oluşuyor

(20800, 5)

In [168]:
news_data.info() # veri seti hakkında bilgi edindim

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [169]:
news_data.columns # kullancagımız veri sütünlarını yazdırdım

Index(['id', 'title', 'author', 'text', 'label'], dtype='object')

In [170]:
news_data.describe().T  # veri setini sekillendirdim

,count,mean,std,min,25%,50%,75%,max
id,20800.0,10399.500000,6004.587135,0.0,5199.75,10399.5,15599.25,20799.0
label,20800.0,0.500625,0.500012,0.0,0.00,1.0,1.00,1.0


In [171]:
print(news_data['label'].value_counts()) # Ayrı ayrı (sahte haber , gercek haber) degerlerini aldım

1    10413
0    10387
Name: label, dtype: int64


In [172]:
news_data.isnull().sum() # eksik degerleri saptamamız gerekiyor

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [173]:
news_data = news_data.fillna('') # Eksik degerleri doldurduk , kontrol edelim ;

In [174]:
news_data.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

Sınıflandırma yapmak için 'Author' ve 'Title' değerlerini kullanacağız.

In [175]:
news_data['content'] = news_data['author']+' '+news_data['title'] # haber baslıgını ve yazarı birleştirdim.

In [176]:
print(news_data['content']) # Şu şekilde gözükecek : 

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799              David Swanson What Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object


Şimdi verileri ve sonuçları(label) ayıracağız bu sayede train-test işlemini yapacağız.

In [177]:
X = news_data.drop(columns='label', axis=1)
Y = news_data['label']

In [178]:
X # Veri setimiz

,id,title,author,text,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,Howard Portnoy Iranian woman jailed for fictio...
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,"Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma..."
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,Michael J. de la Merced and Rachel Abrams Macy...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...","Alex Ansary NATO, Russia To Hold Parallel Exer..."


In [179]:
Y # Label'larımız (sonuçlarımız)

0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 20800, dtype: int64

Stemming İşlemi
Kısacası : Bir kelimeyi köküne indirgemek, ön eki ve son eki kaldırmak.


In [180]:
port_stem = PorterStemmer()

In [181]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [182]:
news_data['content'] = news_data['content'].apply(stemming)

In [183]:
news_data['content']

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object

In [184]:
X = news_data['content'].values
Y = news_data['label'].values

In [185]:
X

array(['darrel lucu hous dem aid even see comey letter jason chaffetz tweet',
       'daniel j flynn flynn hillari clinton big woman campu breitbart',
       'consortiumnew com truth might get fire', ...,
       'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time',
       'alex ansari nato russia hold parallel exercis balkan',
       'david swanson keep f aliv'], dtype=object)

In [186]:
Y

array([1, 0, 1, ..., 0, 1, 1])

Kategorisel veriler calismadigi icin Metinsel verileri sayısal verilere çevirdim

In [187]:
vectorizer = TfidfVectorizer()
vectorizer.fit(X)
X = vectorizer.transform(X)

In [188]:
X

<20800x17128 sparse matrix of type '<class 'numpy.float64'>'
	with 210687 stored elements in Compressed Sparse Row format>

Training - Test işlemi
Verileri eğitim ve test verilerine bölüyoruz ve bu şekilde öğrenme algoritmamızı oluşturuyoruz.

In [189]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2, stratify=Y, random_state=2)

In [190]:
print(X_train)
print("--------------")
print(X_test)
print("--------------")
print(Y_train)
print("--------------")
print(Y_test)

  (0, 16996)	0.09995656818816077
  (0, 16928)	0.3360072514082535
  (0, 15295)	0.09807665903342763
  (0, 13914)	0.3334797245354899
  (0, 13014)	0.2680313811122545
  (0, 12501)	0.3929876463935473
  (0, 11936)	0.24142639024498436
  (0, 10306)	0.09662001419895176
  (0, 10219)	0.3019527708144002
  (0, 3155)	0.3400831511004003
  (0, 2794)	0.3776836172783757
  (0, 336)	0.3360072514082535
  (1, 16996)	0.07263181421455335
  (1, 15424)	0.22579404836928033
  (1, 15417)	0.26613170238131584
  (1, 15295)	0.07126580880898774
  (1, 13453)	0.3387500815971264
  (1, 11421)	0.3084666283145136
  (1, 10306)	0.07020736153621741
  (1, 10061)	0.24924889065491332
  (1, 9856)	0.3387500815971264
  (1, 5313)	0.2768869285533855
  (1, 2761)	0.3387500815971264
  (1, 2207)	0.3387500815971264
  (1, 956)	0.2855580802628186
  :	:
  (16637, 10008)	0.32617532466383187
  (16637, 9588)	0.184164927958451
  (16637, 8153)	0.3214731053599403
  (16637, 5009)	0.3538476463585765
  (16637, 1868)	0.40919228974806576
  (16637, 364)	0.

Lojistik Regresyon Modelinin kurulması


In [191]:
model = LogisticRegression()

In [192]:
model.fit(X_train, Y_train)

LogisticRegression()

Training işlemi , Makine ogrenme zamani

In [193]:
X_train_prediction = model.predict(X_train)

In [194]:
X_train_prediction

array([0, 0, 0, ..., 0, 0, 1])

In [195]:
training_data_accuracy = accuracy_score(X_train_prediction, Y_train) # kesinlik orani

In [196]:
 print('Training verilerinin doğruluk puanı :', training_data_accuracy)

Training verilerinin doğruluk puanı : 0.9865985576923076


Testing işlemi

In [197]:
X_test_prediction = model.predict(X_test)
testing_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [198]:
 print('Test verilerinin doğruluk puanı :', testing_data_accuracy)

Test verilerinin doğruluk puanı : 0.9790865384615385


### ----------------------------------------------------------------------------------
### Şimdi bu veri setini birde Cart (Karar agaci yontemi) ile hesaplayalım..

In [199]:
cart_model = DecisionTreeRegressor()

In [200]:
cart_model

DecisionTreeRegressor()

In [201]:
cart_model.fit(X_train,Y_train)

DecisionTreeRegressor()

In [202]:
cart_X_train_prediction = cart_model.predict(X_train)

In [203]:
cart_X_train_prediction

array([0., 0., 0., ..., 0., 0., 1.])

In [204]:
cart_training_data_accuracy =  accuracy_score(cart_X_train_prediction, Y_train) # dogruluk skoru

In [205]:
 print('Training verilerinin doğruluk puanı :', cart_training_data_accuracy)

Training verilerinin doğruluk puanı : 1.0


#### Testing İşlemi

In [206]:
cart_X_test_prediction = cart_model.predict(X_test)
cart_testing_data_accuracy = accuracy_score(cart_X_test_prediction,Y_test)

In [207]:
print('Test verilerinin doğruluk puanı :', cart_testing_data_accuracy)

Test verilerinin doğruluk puanı : 0.9908653846153846


### ------------------------------------------------------------------

In [208]:
# Lojistik Regresyon Dogruluk Orani : %97.90
# CART Dogruluk Orani : %99.13

In [219]:
### ----------------------------------------------------------------------------------
### Şimdi bu veri setini birde KNN ile hesaplayalım..

In [237]:
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X, Y)
y_pred = knn.predict(X)
print('Test verilerinin doğruluk puanı :', accuracy_score(Y, y_pred))

Test verilerinin doğruluk puanı : 0.5983653846153846


In [ ]:
# Lojistik Regresyon Dogruluk Orani : %97.90
# CART Dogruluk Orani : %99.13
# KNN (n=3) Dogruluk Orani : %59.8